In [76]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import cauchy
from scipy.ndimage import gaussian_filter1d

In [2]:
%matplotlib notebook

In [86]:
def make_data():
    N = 10000
    x = np.linspace(0, 4, N)
    x[2000:] += 1
    x[5500:] -= 2
    x[8000:] += 0.5
    x = gaussian_filter1d(x, 100)
    return x + cauchy.rvs(0, 0.3, N)

In [99]:
x = make_data()

# Running median of a time series

Let $z$ be a time series of real numbers formed by a slowly changing signal $x$ corrupted by additive, independent, and identically distributed noise $\epsilon$, $z(n) = x(n) + \epsilon(n)$. The noise has many outliers, and its mean is undefined, but its median is known to be zero. You also know that $x(n)$ changes slowly; $|x(n) - x(n-1)|$ is usually less than 0.01.

## Question 1

Recover the signal $z$ given $x$ by computing a running estimate $m(n)$ of the **median** of $x(n)$, $m(n) \approx \mathrm{Median}(x(n))$. The estimate must be computed online -- that is, $m(n)$ can only depend on $x$ up to and including index $n$. You estimator may only use a constant (and small) amount of memory; no matter how long $x$ is, the estimator has to use the same amount of memory.

## Question 2

Why does your method work? How does it trade off between precision and fast response?

## Question 3
How might you adapt your method to approximate arbitrary percentiles of $x$?